# Part 1. Benchmarking and Profiling 

1.-  Read the sections “Introducing the Julia Set” and “Calculating the Full Julia Set” on 
Chapter 2. Profiling to Find Bottlenecks from the book: M. Gorelick & I. Ozsvald 
(2020). High Performance Python. Practical Performant Programming for Humans. 
Second Edition. United States of America: O’Reilly Media, Inc. Implement the 
chapter functions (Example 2-1, 2-2, 2-3 and 2-4) on Python in order to calculate the 
Julia Set. Make the representation for the false gray and pure gray scale. 

In [6]:
#2-1
import time
 # area of complex space to investigate
x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_imag = -0.62772, -.42193

In [7]:
#2-2
def calc_pure_python(desired_width, max_iterations):
    x_step = (float(x2 - x1) / float(desired_width))
    y_step = (float(y1 - y2) / float(desired_width))
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
        xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step
    #Built a list of coordinates and the initla condition for each cell 
    #The intial condition is is a constant tha can be removed
    zs = []    
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))

    print("Length of x:", len(x))
    print("Total elements:", len(zs))
    start_time = time.time()
    output = calculate_z_serial_purepython(max_iterations, zs, cs)
    end_time = time.time()
    secs = end_time - start_time
    print(calculate_z_serial_purepython.__name__ + "took", secs, "seconds")
    assert sum(output) == 33219980

In [8]:
#2-3
def calculate_z_serial_purepython(maxiter, zs, cs):
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output


In [9]:
#2-4
if __name__ == "__main__":
    calc_pure_python(desired_width=1000, max_iterations=300)

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepythontook 5.3745200634002686 seconds


### Representation for False Gray and Pure Grayscale

In [10]:
import time
from PIL import Image
import array

In [11]:
def show_false_greyscale(output_raw, width, height, max_iterations):
    """Convert list to array, show using PIL"""
    # convert our output to PIL-compatible input
    assert width * height == len(output_raw)  
    max_value = float(max(output_raw))
    output_raw_limited = [int(float(o) / max_value * 255) for o in output_raw]
    output_rgb = ((o + (256 * o) + (256 ** 2) * o) * 16 for o in output_raw_limited)
    output_rgb = array.array('I', output_rgb) 
    # display with PIL/pillow
    im = Image.new("RGB", (width, height))
    
    im.frombytes(output_rgb.tobytes(), "raw", "RGBX", 0, -1)
    im.show()

In [12]:
def show_greyscale(output_raw, width, height, max_iterations):
    """Convert list to array, show using PIL"""
    # convert our output to PIL-compatible input
    max_iterations = float(max(output_raw))
    print(max_iterations)
    scale_factor = float(max_iterations)
    scaled = [int(o / scale_factor * 255) for o in output_raw]
    output = array.array('B', scaled)  
    # display with PIL
    im = Image.new("L", (width, width))
    
    im.frombytes(output.tobytes(), "raw", "L", 0, -1)
    im.show()

In [13]:
def calc_pure_python(draw_output, desired_width, max_iterations):
    """Create a list of complex co-ordinates (zs) and complex parameters (cs), build Julia set and display"""
    x_step = (x2 - x1) / desired_width
    y_step = (y1 - y2) / desired_width
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step
    # set width and height to the generated pixel counts, rather than the
    # pre-rounding desired width and height
    width = len(x)
    height = len(y)
    # build a list of co-ordinates and the initial condition for each cell.
    # Note that our initial condition is a constant and could easily be removed,
    # we use it to simulate a real-world scenario with several inputs to our function
    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))

    print("Length of x:", len(x))
    print("Total elements:", len(zs))
    start_time = time.time()
    output = calculate_z_serial_purepython(max_iterations, zs, cs)
    end_time = time.time()
    secs = end_time - start_time
    print(calculate_z_serial_purepython.__name__ + " took", secs, "seconds")

    assert sum(output) == 33219980  # this sum is expected for 1000^2 grid with 300 iterations

    if draw_output:
        show_greyscale(output, width, height, max_iterations)

In [14]:
if __name__ == "__main__":
    # Calculate the Julia set using a pure Python solution with
    # reasonable defaults for a laptop
    # set draw_output to True to use PIL to draw an image
    calc_pure_python(draw_output=True, desired_width=1000, max_iterations=300)

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 5.177051544189453 seconds
300.0


2.- Define a new function, timefn, which takes a function as an argument: the inner 
function, measure_time, takes *args (a variable number of positional arguments) 
and **kwargs (a variable number of key/value arguments) and passes them through 
to fn for execution. Decorate calculate_z_serial_purepython with @timefn to profile 
it. Implement Example 2-5 and adapt your current source code. 

In [15]:
from functools import wraps
def timefn(fn):
    @wraps(fn)
    def measure_time(*args, **kwargs):
        t1 = time.time()
        result = fn(*args, **kwargs)
        t2 = time.time()
        print("@timefn:" + fn.__name__ + " took " + str(t2 - t1) + " seconds")
        return result
    return measure_time


In [17]:
@timefn
def calculate_z_serial_purepython(maxiter, zs, cs):
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output

In [19]:
calc_pure_python(draw_output=False, desired_width=1000, max_iterations=300)

Length of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 5.191513776779175 seconds
calculate_z_serial_purepython took 5.192037343978882 seconds


3.- Use the timeit modeule to get a coarse measurement of the execution speed of the 
CPU-bound function. Runs 10 loops with 5 repetitions. Show how to do the 
measurement on the command line and on a Jupyter Notebook. 

In [24]:
%timeit -r 5 -n 10 calc_pure_python(draw_output=False,desired_width=1000, max_iterations=300)

Length of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 5.240819931030273 seconds
calculate_z_serial_purepython took 5.2409467697143555 seconds
Length of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 5.269282341003418 seconds
calculate_z_serial_purepython took 5.269427061080933 seconds
Length of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 5.164397478103638 seconds
calculate_z_serial_purepython took 5.164538383483887 seconds
Length of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 5.23262357711792 seconds
calculate_z_serial_purepython took 5.23275351524353 seconds
Length of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 5.479423522949219 seconds
calculate_z_serial_purepython took 5.479592323303223 seconds
Length of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 5.226547002792358 seconds
calculate_z_serial_purepython 

4.- Use the cProfile module to profile the source code (.py). Sort the results by the time 
spent inside each function. This will give a view into the slowest parts. Analyze the 
output and make a syntesis of the findings. Show how to use the cProfile module on 
the command line and on a Jupyter Notebook. 

In [23]:
import cProfile
import pstats

In [25]:
profiler = cProfile.Profile()
profiler.enable()
calc_pure_python(draw_output=False, desired_width=1000, max_iterations=300)
profiler.disable()

Length of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 7.8562846183776855 seconds
calculate_z_serial_purepython took 7.856455564498901 seconds


In [26]:
stats = pstats.Stats(profiler).sort_stats('cumulative')
stats.print_stats()

         36222268 function calls in 8.597 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    8.597    4.298 /home/samantha/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3302(run_code)
        2    0.000    0.000    8.596    4.298 {built-in method builtins.exec}
        1    0.036    0.036    8.596    8.596 <ipython-input-25-a151f6932651>:3(<module>)
        1    0.545    0.545    8.561    8.561 <ipython-input-13-b8e84de4a7ff>:1(calc_pure_python)
        1    0.000    0.000    7.856    7.856 <ipython-input-15-722e9f04643c>:3(measure_time)
        1    5.902    5.902    7.856    7.856 <ipython-input-17-57ebe2720ed6>:1(calculate_z_serial_purepython)
 34219980    1.954    0.000    1.954    0.000 {built-in method builtins.abs}
  2002000    0.150    0.000    0.150    0.000 {method 'append' of 'list' objects}
        1    0.005    0.005    0.005    0.005 {built-in method builtins

5.- Use snakeviz to get a high-level understanding of the cPrifile statistics file. Analyze 
the output and make a syntesis of the findings. 

In [27]:
%reload_ext snakeviz

if __name__ == "__main__":
    cProfile.run('calc_pure_python(draw_output=False, desired_width=1000, max_iterations=300)', 'stats')
!snakeviz "stats"

Length of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 7.893105506896973 seconds
calculate_z_serial_purepython took 7.893787145614624 seconds
snakeviz web server started on 127.0.0.1:8080; enter Ctrl-C to exit
http://127.0.0.1:8080/snakeviz/%2Fmnt%2Fc%2FUsers%2Fs_cas%2FOneDrive%2FDocumentos%2FUPY%2FCuatri%207%2Fhpc%2FCA3%2F1%2Fstats


6.- Use the line_profiler and kernprof file to profile line-by-line the function 
calculate_z_serial_purepython. Analyze the output and make a syntesis of the 
findings. 

In [30]:
%load_ext line_profiler
%lprun -f calculate_z_serial_purepython calc_pure_python(draw_output=False, desired_width=1000, max_iterations=300)

/home/samantha/.local/lib/python3.6/site-packages/line_profiler/ipython_extension.py:97: UserWarning: Adding a function with a __wrapped__ attribute. You may want to profile the wrapped function by adding calculate_z_serial_purepython.__wrapped__ instead.
  profile = LineProfiler(*funcs)


Length of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 12.422465801239014 seconds
calculate_z_serial_purepython took 12.42318844795227 seconds


Timer unit: 1e-09 s

Total time: 12.4227 s
File: <ipython-input-15-722e9f04643c>
Function: measure_time at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                               @wraps(fn)
     4                                               def measure_time(*args, **kwargs):
     5         1       3300.0   3300.0      0.0          t1 = time.time()
     6         1        1e+10    1e+10    100.0          result = fn(*args, **kwargs)
     7         1      18900.0  18900.0      0.0          t2 = time.time()
     8         1     250600.0 250600.0      0.0          print("@timefn:" + fn.__name__ + " took " + str(t2 - t1) + " seconds")
     9         1        200.0    200.0      0.0          return result

7.- Use the memory_profiler to diagnose memory usage. Analyze the output and make 
a syntesis of the findings. 

In [36]:
%reload_ext memory_profiler
%memit calc_pure_python(draw_output=False, desired_width=1000, max_iterations=300)

Length of x: 1000
Total elements: 1000000
@timefn:calculate_z_serial_purepython took 5.769466876983643 seconds
calculate_z_serial_purepython took 5.770303010940552 seconds
peak memory: 179.38 MiB, increment: 80.73 MiB
